In [1]:
import numpy as np

# --- Helper to handle N-qubit Kronecker products easily ---
from functools import reduce

def kron_n(matrices):
    """
    Helper function to Kronecker product a list of matrices.
    e.g., kron_n([H, H, H]) results in H (x) H (x) H
    """
    return reduce(np.kron, matrices)

def get_initial_superposition(ket_0, h_gate, n_qubits):
    """
    Step 1: Initialize to equal superposition |s>.
    """
    # Create list of |0> and H gates based on n_qubits
    kets = [ket_0] * n_qubits
    gates = [h_gate] * n_qubits

    # |000>
    psi_0 = kron_n(kets)

    # H_total (8x8 matrix for 3 qubits)
    h_total = kron_n(gates)

    # |s> = H_total * |000>
    psi_s = h_total @ psi_0

    print(f"Initial Superposition (First 5 elements): {psi_s.flatten()[:5].round(3)}...")
    return psi_s

def create_oracle_matrix(target_index, n_qubits):
    """
    Step 2: The Oracle U_w.
    Matrix size is 2^n x 2^n.
    """
    N = 2**n_qubits
    U_w = np.eye(N)

    # Flip the sign of the target element
    U_w[target_index, target_index] = -1
    return U_w

def create_diffuser_matrix(n_qubits):
    """
    Step 3: The Diffuser.
    D = 2|s><s| - I
    """
    N = 2**n_qubits

    # Create |s> vector dynamically (value is 1/sqrt(N))
    # For 3 qubits, 1/sqrt(8) approx 0.353
    s_vector = np.ones((N, 1)) / np.sqrt(N)

    # Outer product |s><s|
    s_outer_s = s_vector @ s_vector.T

    # D = 2 * (|s><s|) - I
    diffuser = 2 * s_outer_s - np.eye(N)
    return diffuser

def measure_state(psi):
    probs = np.abs(psi.flatten())**2
    return probs

def run_grover_simulation(target_name, target_index, ket_0, h_gate):
    n_qubits = 3 # <--- CHANGED TO 3

    print(f"\n{'='*60}")
    print(f"--- Searching for Target: {target_name} (Index {target_index}) ---")

    # 1. Initialization
    psi = get_initial_superposition(ket_0, h_gate, n_qubits)

    # Generate Operators
    oracle = create_oracle_matrix(target_index, n_qubits)
    diffuser = create_diffuser_matrix(n_qubits)

    # --- THE LOOP ---
    # For N=8, we need roughly 2 iterations to maximize probability.
    # If we only run it once, probability is only ~78%.
    optimal_iterations = 2

    for i in range(optimal_iterations):
        print(f"\n[Iteration {i+1}]")

        # Apply Oracle
        psi = oracle @ psi
        # Apply Diffuser
        psi = diffuser @ psi

        # Peek at the probability of the winner
        current_probs = measure_state(psi)
        print(f"   Current prob of target: {current_probs[target_index]*100:.1f}%")

    # 4. Final Measurement
    probs = measure_state(psi)
    measured_index = np.argmax(probs)

    print(f"\nFinal Probabilities (Rounded):\n{probs.round(3)}")
    print(f">> FOUND: Index {measured_index} with probability {probs[measured_index]*100:.1f}%")

    if measured_index == target_index:
        print(">> SUCCESS")
    else:
        print(">> FAILURE")

def main():
    # Standard Definitions
    ket_0 = np.array([[1], [0]])
    h_gate = (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]])

    # Targets for 3 Qubits (Indices 0 to 7)
    # |111> is index 7 (Binary 111 = 7)
    # |101> is index 5 (Binary 101 = 5)
    targets = [
        ("|111>", 7),
        ("|101>", 5)
    ]

    for name, idx in targets:
        run_grover_simulation(name, idx, ket_0, h_gate)

if __name__ == "__main__":
    main()


--- Searching for Target: |111> (Index 7) ---
Initial Superposition (First 5 elements): [0.354 0.354 0.354 0.354 0.354]...

[Iteration 1]
   Current prob of target: 78.1%

[Iteration 2]
   Current prob of target: 94.5%

Final Probabilities (Rounded):
[0.008 0.008 0.008 0.008 0.008 0.008 0.008 0.945]
>> FOUND: Index 7 with probability 94.5%
>> SUCCESS

--- Searching for Target: |101> (Index 5) ---
Initial Superposition (First 5 elements): [0.354 0.354 0.354 0.354 0.354]...

[Iteration 1]
   Current prob of target: 78.1%

[Iteration 2]
   Current prob of target: 94.5%

Final Probabilities (Rounded):
[0.008 0.008 0.008 0.008 0.008 0.945 0.008 0.008]
>> FOUND: Index 5 with probability 94.5%
>> SUCCESS
